In [1]:
!pip install xbbg pandas numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/27.5 MB ? eta -:--:--
   --------------------------------- ------ 22.8/27.5 MB 111.1 MB/s eta 0:00:01
   ---------------------------------------- 27.5/27.5 MB 91.8 MB/s eta 0:00:00


In [3]:
import sys
print(sys.executable)

!{sys.executable} -m pip install --index-url=https://blpapi.bloomberg.com/repository/releases/python/simple blpapi


c:\ProgramData\anaconda3\envs\PYTHON_3.11\python.exe
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://blpapi.bloomberg.com/repository/releases/python/simple
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 5.6/5.6 MB 84.7 MB/s eta 0:00:00


In [4]:
!pip install XlsxWriter

Defaulting to user installation because normal site-packages is not writeable


In [16]:
!pip install pandas_market_calendars

Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [19]:
"""
Fetch SPX universe (monthly trading month-end NYSE) + members PX_LAST → Excel (2 sheets)
- Dates = dernier jour de trading NYSE de chaque mois (évite week-ends / jours fériés)
- Colonne ticker = "AAPL UW Equity" etc.
- Colonne PX_LAST = dernier prix dispo <= date (lookback)
"""

import pandas as pd
from pathlib import Path
from xbbg import blp
import pandas_market_calendars as mcal


def get_members(index_ticker, date, add_suffix=" Equity"):
    dt = date.replace("-", "")
    raw = blp.bds(index_ticker, "INDX_MWEIGHT_HIST", END_DATE_OVERRIDE=dt)

    if not isinstance(raw, pd.DataFrame) or raw.empty:
        return []

    cols = {str(c).strip().lower(): c for c in raw.columns}

    preferred_keys = [
        "member_ticker_and_exchange_code",
        "index_member",
        "member",
    ]
    tcol = next((cols[k] for k in preferred_keys if k in cols), None)
    if not tcol:
        return []

    s = raw[tcol].astype(str).str.strip()
    s = s[s.notna() & (s != "") & (s.str.lower() != "nan")]

    # Enlève les IDs numériques (Bloomberg IDs)
    s = s[~s.str.match(r"^\d")]

    # Garde uniquement "TICKER EXCH" (ex: AAPL UW)
    s = s[s.str.match(r"^[A-Z0-9./-]+ [A-Z]{1,4}$")]

    # Ajoute " Equity" partout (sans doublonner si déjà présent)
    s = s.apply(lambda x: x if x.upper().endswith(" EQUITY") else x + add_suffix)

    # Dédoublonne en conservant l'ordre
    seen = set()
    out = []
    for x in s.tolist():
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out


def _chunk(lst, n=150):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


def get_members_px_last_asof(tickers, asof_date, lookback_days=10):
    """
    Retourne un dict {ticker: px_last} au plus proche <= asof_date.
    On récupère une petite plage [asof-lookback, asof] et on prend la dernière valeur dispo.
    Version robuste: ne dépend pas de df.xs("PX_LAST").
    """
    if not tickers:
        return {}

    end = pd.to_datetime(asof_date)
    start = end - pd.Timedelta(days=lookback_days)

    start_s = start.strftime("%Y-%m-%d")
    end_s = end.strftime("%Y-%m-%d")

    out = {}

    for chunk in _chunk(tickers, n=150):
        df = blp.bdh(
            tickers=chunk,
            flds="PX_LAST",
            start_date=start_s,
            end_date=end_s,
        )

        if df is None or not isinstance(df, pd.DataFrame) or df.empty:
            for t in chunk:
                out[t] = float("nan")
            continue

        # --- Cas multi-tickers : colonnes MultiIndex (ticker, field)
        if isinstance(df.columns, pd.MultiIndex):
            lvl1 = df.columns.get_level_values(1).astype(str).str.strip().str.upper()
            mask = (lvl1 == "PX_LAST")

            if not mask.any():
                for t in chunk:
                    out[t] = float("nan")
                continue

            px = df.loc[:, mask].copy()
            px.columns = px.columns.get_level_values(0)

            for t in chunk:
                if t in px.columns:
                    s = px[t].dropna()
                    out[t] = float(s.iloc[-1]) if len(s) else float("nan")
                else:
                    out[t] = float("nan")
        else:
            # --- Cas 1 seul ticker (ou structure non MultiIndex)
            col0 = df.columns[0]
            s = df[col0].dropna()
            out[chunk[0]] = float(s.iloc[-1]) if len(s) else float("nan")

    return out


def get_monthly_nyse_trading_month_ends(start_date: str, end_date: str):
    """
    Renvoie une liste de dates (YYYY-MM-DD) correspondant au dernier jour de trading NYSE
    pour chaque mois entre start_date et end_date.
    """
    nyse = mcal.get_calendar("NYSE")
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)

    trading_days = schedule.index  # DatetimeIndex (jours de bourse)
    monthly_trading_days = trading_days.to_series().groupby(trading_days.to_period("M")).max()

    return monthly_trading_days.dt.strftime("%Y-%m-%d").tolist()


def get_spx_px_last_on_dates(dates_yyyy_mm_dd):
    """
    Récupère SPX PX_LAST uniquement pour les dates données (trading month-ends NYSE).
    Utilise un lookback pour être robuste.
    """
    if not dates_yyyy_mm_dd:
        return pd.DataFrame(columns=["date", "spx_px_last"])

    # On prend une plage large, puis on filtre (plus simple & robuste)
    start = dates_yyyy_mm_dd[0]
    end = dates_yyyy_mm_dd[-1]

    df = blp.bdh(
        tickers="SPX Index",
        flds="PX_LAST",
        start_date=start,
        end_date=end,
    )

    s = df.iloc[:, 0].copy()
    s.index = pd.to_datetime(s.index)

    wanted = pd.to_datetime(pd.Series(dates_yyyy_mm_dd))
    # pour chaque date voulue, on prend le dernier prix dispo <= date
    out = []
    for d in wanted:
        sub = s.loc[:d].dropna()
        out.append(float(sub.iloc[-1]) if len(sub) else float("nan"))

    df_out = pd.DataFrame({"date": wanted.dt.strftime("%Y-%m-%d"), "spx_px_last": out})
    return df_out


# =========================
# --- Paramètres
# =========================
start = "2015-01-01"
end = "2025-12-31"

# Dates = dernier jour de trading NYSE de chaque mois
dates = get_monthly_nyse_trading_month_ends(start, end)

rows = []

# --- Universe extraction + PX_LAST des membres
for i, d in enumerate(dates):
    print(f"[{i+1}/{len(dates)}] {d}...", end="", flush=True)

    members = get_members("SPX Index", d)
    print(f" members={len(members)}", end="")

    px_map = get_members_px_last_asof(members, d, lookback_days=10)
    print(" ✓")

    for t in members:
        rows.append({"date": d, "ticker": t, "PX_LAST": px_map.get(t)})

df_universe = pd.DataFrame(rows)

# Format date pour Excel (affichage)
df_universe["date"] = pd.to_datetime(df_universe["date"]).dt.strftime("%d/%m/%Y")

# --- SPX PX_LAST extraction sur les mêmes dates
df_spx = get_spx_px_last_on_dates(dates)
df_spx["date"] = pd.to_datetime(df_spx["date"]).dt.strftime("%d/%m/%Y")

# --- Export Excel (2 sheets)
out = Path.home() / "Desktop" / "SPX_universe_with_price_tes_date2.xlsx"

with pd.ExcelWriter(out, engine="xlsxwriter") as writer:
    df_universe.to_excel(writer, sheet_name="SPX_Components", index=False)
    df_spx.to_excel(writer, sheet_name="SPX_PX_LAST", index=False)

print(f"\n✅ File saved to {out}")
print(f"Components rows: {len(df_universe):,}")
print(f"SPX monthly points: {len(df_spx):,}")

[1/132] 2015-01-30... members=466 ✓
[2/132] 2015-02-27... members=466 ✓
[3/132] 2015-03-31... members=467 ✓
[4/132] 2015-04-30... members=467 ✓
[5/132] 2015-05-29... members=467 ✓
[6/132] 2015-06-30... members=468 ✓
[7/132] 2015-07-31... members=469 ✓
[8/132] 2015-08-31... members=471 ✓
[9/132] 2015-09-30... members=474 ✓
[10/132] 2015-10-30... members=474 ✓
[11/132] 2015-11-30... members=475 ✓
[12/132] 2015-12-31... members=475 ✓
[13/132] 2016-01-29... members=476 ✓
[14/132] 2016-02-29... members=477 ✓
[15/132] 2016-03-31... members=478 ✓
[16/132] 2016-04-29... members=480 ✓
[17/132] 2016-05-31... members=482 ✓
[18/132] 2016-06-30... members=482 ✓
[19/132] 2016-07-29... members=483 ✓
[20/132] 2016-08-31... members=483 ✓
[21/132] 2016-09-30... members=485 ✓
[22/132] 2016-10-31... members=486 ✓
[23/132] 2016-11-30... members=485 ✓
[24/132] 2016-12-30... members=485 ✓
[25/132] 2017-01-31... members=485 ✓
[26/132] 2017-02-28... members=486 ✓
[27/132] 2017-03-31... members=486 ✓
[28/132] 2